# 1. Linear regression model

First we implement the linear regression model.

In [60]:
import numpy as np
import pandas as pd

In [141]:
class Linear_regression():
  #Linear regression

  def __init__(self,W=None):
    #W = np.array of size (p,1)


    #We will rescale the features to have mean 0 and std 1
    self.scaler = StandardScaler()

    #weights
    self.W=W
    #number of features
    self.p = None

    #for our convenience, we keep track of the standard deviation, and z-score in the model
    self.stderr = None
    self.z_score = None
    
  def fit(self, X, y):
    # X = np.array of size (n,p)
    # y = np.array of size (n,)

    
    x = self.scaler.fit_transform(X)
    n,p = x.shape
    self.p = p+1

    # We add the constant feature to X:
    x=np.append(np.ones(shape=(n,1)),x,axis=1).reshape((n,p+1))

    XXi = inv((x.T @ x))

    self.W = XXi @ x.T @ y
    # W has shape (p,n)@(n,p)@(p,n)@(n,1)=(p,1)
    
    sigma = (((x @ self.W - y) @ (x @ self. W - y))/(n - p - 1))**0.5

    #compute the stderr and z-score while training the model
    self.stderr = (np.diag(XXi))**0.5 * sigma
    self.z_score = self.W / self.stderr



  def predict(self, X):
    x = self.scaler.transform(X)
    n=x.shape[0]
    x=np.append(np.ones(shape=(n,1)),x,axis=1).reshape((n,self.p))
    
    return x @ self.W
    # return has shape (n,1)

  def score(self, X, y):
    #MSE score
    return (self.predict(X) - y) @ (self.predict(X) - y) / y.size

  def accuracy(self, X, y):
    #accuracy, where we classify the label as 1 if y>=0.5, and 0 otherwise
    return ((self.predict(X)>=0.5) == y).sum()/ y.size

# 2. Linear Regression Analysis of Spam Dataset

We use the Spam dataset available here https://hastie.su.domains/ElemStatLearn/.

There are 56 unnamed features of dtype float, and a label indicating if the mail is spam or not.

In [142]:
df = pd.read_csv("/content/spam.data", sep='\s+', header = None,  skipinitialspace=True)

In [105]:
df

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.000,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.010,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,0.31,0.00,0.62,0.0,0.00,0.31,0.00,0.00,0.00,0.00,...,0.000,0.232,0.0,0.000,0.000,0.000,1.142,3,88,0
4597,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.353,0.000,0.000,1.555,4,14,0
4598,0.30,0.00,0.30,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.102,0.718,0.0,0.000,0.000,0.000,1.404,6,118,0
4599,0.96,0.00,0.00,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.057,0.0,0.000,0.000,0.000,1.147,5,78,0


Split the train and test data:

In [143]:
traintest = pd.read_csv("/content/spam.traintest",  sep='\s+', header = None,  skipinitialspace=True)
df['train'] = traintest
df_train = df[df.train==1]
df_test = df[df.train == 0]

X_train = df_train.iloc[:, :57]
y_train = df_train[57]


X_test = df_test.iloc[:, :57]
y_test = df_test[57]

Train the model:

In [145]:
lin_reg = Linear_regression()
lin_reg.fit(X_train, y_train)
y_pred = lin_reg.predict(X_train)

# 3. Dropping features

In the table below we compute the Z-coefficients for the features:

In [146]:
table = pd.DataFrame(data = {'Coefficient' : lin_reg.W, 'Std. Error': lin_reg.stderr, 'Z Score': lin_reg.z_score}, index = ['Intercept'] + list(df.columns)[:-2])
# table.sort_values('Abs Z Score')
table.head()

,Coefficient,Std. Error,Z Score
Intercept,0.387370,0.008320,46.558006
0,-0.028806,0.009483,-3.037660
1,-0.010696,0.008533,-1.253428
2,0.006182,0.008876,0.696476
3,0.015095,0.008385,1.800189


A Z-score greater than 2 in absolute value is significant at the 5% level, meaning that we can consider dropping all the features with absolute value below 2.

First we create the list of such features:

In [156]:
drop_features = list(table[table['Z Score'].map(abs) < 2].index)
print("Number of potentially insignificant features:", len(drop_features))

Number of potentially insignificant features: 24


Let us compute the F-statistic for this situation:
```
F = (RSS0 - RSS1)/(p1 - p0) / (RSS1/(N-p1-1))
```
where RSS1 is the sum of the squared errors for our model, and RSS0 is the same for the smaller model with dropped features,
and p1 and p0 are the numbers of features in the two models.




In [148]:
RSS1 = (y_pred - y_train) @ (y_pred - y_train)

To compute RSS0 we need to train the smaller model:

In [149]:
lin_reg_2 = Linear_regression()

X_train_2 = X_train.drop(columns = drop_features)

lin_reg_2.fit(X_train_2, y_train)

y_pred_2 = lin_reg_2.predict(X_train_2)

RSS0 = (y_pred_2 - y_train) @ (y_pred_2 - y_train)

Now we are ready to compute the F statistic:

In [150]:
Fstat = (RSS0 - RSS1)/(len(drop_features))/RSS1 * (y_train.size - X_train.shape[1]-1)
Fstat

1.4870410790131892

To check if the F statistic is significant on the 5% level, we need the F distributions with parameters dfn = p1 - p0, and dfd = N - p1 - 1

In [151]:
from scipy.stats import f
dfn, dfd =len(drop_features), 58

f.cdf(1.48, dfn, dfd, loc=0, scale=1)

0.8869462397610554

This implies that the F statistic is not significant, and we may drop the features.

# 3. Comparing the Models

We evaluate the models, first with respect to the MSE (mean squared error), and then the accuracy of the model.

In [153]:
print("The first model score = ", lin_reg.score(X_test, y_test))
print("The secon model score = ", lin_reg_2.score(X_test.drop(columns = drop_features), y_test))

The first model score =  0.1238651160906676
The secon model score =  0.11867131055900887


In [154]:
print("The first model accuracy = ", lin_reg.accuracy(X_test, y_test))
print("The secon model accuracy = ", lin_reg_2.accuracy(X_test.drop(columns = drop_features), y_test))

The first model accuracy =  0.8884176182707993
The secon model accuracy =  0.8845024469820555


The two models perform almost the same, so we prefer the second model because it is simpler.

Finally, we recompute the Z-coefficients to see if there are any other insignificant features.

In [163]:
table2 = pd.DataFrame(data = {'Coefficient' : lin_reg_2.W, 'Std. Error': lin_reg_2.stderr, 'Z Score': lin_reg_2.z_score}, index = ['Intercept'] + list(X_train_2.columns))
table2.head()

,Coefficient,Std. Error,Z Score
Intercept,0.387370,0.008352,46.377893
0,-0.025898,0.009422,-2.748554
4,0.074657,0.009074,8.227202
5,0.026198,0.008753,2.992923
6,0.084796,0.008755,9.685118


In [166]:
list(table2[table2['Z Score'].map(abs) < 2].index)

[]